# How ChatGPT Works Part 1: Supervised Fine-Tuning

## Collecting Prompts

Submit some prompts [here](https://docs.google.com/forms/d/e/1FAIpQLSfSBFzS4yrdUwy3DjEj2kskTc1JXk-T47TbmK8TaEgSt4fkcA/viewform?usp=sf_link).

## Collecting Ideal Prompt Responses

These days, there are more sophisticated ways to manage data collection.
One platform is [Scale AI](https://scale.com).

Submit your email [here](https://docs.google.com/forms/d/e/1FAIpQLScsMfW1Fh0bwget7cZKCzm6TQ-1c0AsvQFHtBain2l1mjnIcQ/viewform?usp=sf_link) so I can allow you to join my demo labelling workforce and you can access the Scale labelling platform.



In [1]:
import torch
import json

class SFTDataset(torch.utils.data.Dataset):
    def __init__(self):
        with open("raw_data.json") as f:
            self.data = json.load(f)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]["prompt"], self.data[idx]["response"]
    

dataset = SFTDataset()
print(dataset[0])

('The best reasons to study at Oxford university:\n- Full of culture\n- Attracts great people\n- Excellent staff', '- Every college has its own bar and drink')


In [ ]:
class 